In [1]:
#imports
import boto3
import pyodbc
import pandas as pd
import redshift_connector
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import os
import json
from functools import reduce

In [2]:
with open('aws.json') as json_file:
    cr = json.load(json_file)

s3creds = cr['data_sources']['s3']['creds']
redcreds = cr['data_sources']['redshift']['creds']

In [3]:
KEYPATH = "gcp.json"
CREDS = service_account.Credentials.from_service_account_file(KEYPATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)


q1 = """
SELECT * FROM training_dataset.tournament
JOIN training_dataset.tournament_winner ON tournament_winner.row_order = tournament.row_order
WHERE tournament.Round = '4th Round'
"""

view1 = client.query(q1)

tournamentR4 = view1.result().to_dataframe()

In [4]:
#s3 connect 
s3 = boto3.resource(
    service_name='s3',
    region_name=s3creds['region_name'],
    aws_access_key_id=s3creds['aws_access_key_id'],
    aws_secret_access_key=s3creds['aws_secret_access_key']
)

tw_name = 'Tournament_Won.csv'
tl_name = 'Tournament_Lost.csv'

#s3 get data
bucket_name = s3creds['bucket']
tw_file_name = s3creds['folder_path']+tw_name
tl_file_name = s3creds['folder_path']+tl_name
bucket = s3.Bucket(bucket_name)
tw_obj = bucket.Object(key=tw_file_name)
tl_obj = bucket.Object(key=tl_file_name)
tw_response = tw_obj.get()
tl_response = tl_obj.get()
twTable = pd.read_csv(tw_response['Body'],header=3)
tlTable = pd.read_csv(tl_response['Body'],header=3)

In [5]:
#connecting to redshift using redshift connector

HOST = redcreds['host']
DATABASE = redcreds['database']
USER = redcreds['username']
PASSWORD = redcreds['password']

conn = redshift_connector.connect(
     host=HOST,
     database=DATABASE,
     user=USER,
     password=PASSWORD
  )

print(conn)
cursor = conn.cursor()

query = "select * from tournament_loser"
cursor.execute(query)
tournament_loser : pd.DataFrame = cursor.fetch_dataframe()

In [6]:
dataFrames = [tournamentR4,twTable,tlTable,tournament_loser]

In [7]:
merged = reduce(lambda left,right: pd.merge(left,right,on='row_order'),dataFrames)

In [10]:
stdCols = ['row_order','Round','Winner','Loser','pl1_year_pro','pl2_year_pro']
mergedStd = merged[stdCols]
mergedStd

,row_order,Round,Winner,Loser,pl1_year_pro,pl2_year_pro
0,12343,4th Round,Schwartzman D.,Sonego L.,2010,2013.0
1,16698,4th Round,Schwartzman D.,Struff J.L.,2010,2009.0
2,18973,4th Round,Schwartzman D.,Anderson K.,2010,2007.0
3,29783,4th Round,Schwartzman D.,Zverev A.,2010,2013.0
4,30629,4th Round,Schwartzman D.,Pouille L.,2010,2012.0
...,...,...,...,...,...,...
651,10846,4th Round,Isner J.,Cilic M.,2007,2005.0
652,11138,4th Round,Isner J.,Raonic M.,2007,2008.0
653,23281,4th Round,Isner J.,Tsitsipas S.,2007,2016.0
654,28660,4th Round,Isner J.,Simon G.,2007,2002.0


In [23]:
players = []
for player in mergedStd[mergedStd.pl1_year_pro==2007]['Winner'].unique():
    players.append(player)
for player in mergedStd[mergedStd.pl2_year_pro==2007.0]['Loser'].unique():
    players.append(player)
print(len(players))

18


In [24]:
data = {'Players':players}
uploadData = pd.DataFrame(data)

In [27]:
pandas_gbq.to_gbq(uploadData, destination_table = 'training_dataset.DA-14', project_id=CREDS.project_id, credentials = CREDS,if_exists='replace')